In [28]:
import ee
import folium
import geehydro
import geemap

In [29]:
ee.Initialize(project='ml-meets-the-sun')

In [30]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [31]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [32]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [33]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [34]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [35]:
era5_2mt_temp = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .select('temperature_2m')
    .filter(ee.Filter.date('2020-01-01', '2025-07-31'))
)
display(era5_2mt_temp)

In [36]:
era5_2mt_dew_point_temp = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .select('dewpoint_temperature_2m')
    .filter(ee.Filter.date('2020-01-01', '2025-07-31'))
)
display(era5_2mt_dew_point_temp) 

In [37]:
era5_2mt_dew_point_temp.first()

In [38]:
joined = ee.ImageCollection(
    ee.Join.inner().apply(
        era5_2mt_temp, era5_2mt_dew_point_temp,
        ee.Filter.equals(
            leftField='system:time_start',
            rightField='system:time_start'
        )
    )
)

In [39]:
joined.first()

In [40]:
def compute_relative_humidity(img):
    temp_c = ee.Image(img.get('primary')).subtract(273.15)
    temp_dew_c = ee.Image(img.get('secondary')).subtract(273.15)

    relative_humidity = ee.Image.exp(
        temp_dew_c.multiply(17.625).divide(temp_dew_c.add(243.04))
    ).divide(
        ee.Image.exp(
            temp_c.multiply(17.625).divide(temp_c.add(243.04))
        )
    ).multiply(100)

    return relative_humidity.copyProperties(
        ee.Image(img.get('primary')),
        ['system:time_start']
    )

rh_collection = joined.map(compute_relative_humidity)


In [41]:
rh_first = rh_collection.first()

In [42]:
mean_relative_humidity= rh_collection.mean().setDefaultProjection(
    rh_collection.first().projection()
)


In [43]:
mean_relative_humidity.projection()

In [44]:
result_clip = {}

def mean_rh_clip(state, name):
    cname = name + "_rh"
    clipped = mean_relative_humidity.clip(state)
    result_clip[cname] = clipped

In [45]:
for name,state in states.items():
    mean_rh_clip(state, name)

In [46]:
result_clip

{'Karnataka_rh': <ee.image.Image at 0x1dda3bfc400>,
 'Tamil Nadu_rh': <ee.image.Image at 0x1dda3bfc100>,
 'Rajasthan_rh': <ee.image.Image at 0x1dda3bfc040>}

In [55]:
def display_rh(cname, clipped):
    
    vis_rh ={
    'min': 0,
    'max': 100,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
    Map.addLayer(clipped,  vis_rh, cname, True, 0.5)
    return Map

In [56]:
display_rh(cname, clipped)

Map(bottom=490075.0, center=[11.765528597430068, 65.60726165771486], controls=(WidgetControl(options=['positio…

In [48]:
 vis_rh ={
    'min': 250.0,
    'max': 320.0,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
        
    ]
    }
Map.addLayer(era5_2mt_dew_point_temp,  vis_rh, 'cname', True, 0.5)

In [49]:
Map

Map(bottom=4043.0, center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], …

In [50]:
for cname,clipped in result_clip.items():
    display_rh(cname, clipped)

In [51]:
mean_relative_humidity.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=karnataka,
    scale=10000,
    maxPixels=1e13
).getInfo()


{'dewpoint_temperature_2m_max': 84.46699848171889,
 'dewpoint_temperature_2m_min': 55.778420410862886}

In [52]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_rh'],
    description='KA_rh_mean',
    assetId='projects/ml-meets-the-sun/assets/KA_rh_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [53]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_rh'],
    description='TN_rh_mean',
    assetId='projects/ml-meets-the-sun/assets/TN_rh_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [54]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_rh'],
    description='RA_rh_mean',
    assetId='projects/ml-meets-the-sun/assets/RA_rh_mean',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()